# Event by Event Data Analysis
## Debugging and looping with a MiniDst.
The code below allows one to look at the structure of individual events, 
and make simple loops through the data. Such loops are slower, but can help find errors.

It also shows how you can look at individual events. 

For the Monte Carlo (MC) data, the partices in the events are known, because they were "thrown" into the
detector. This information is stored in the mc_particle* leaves of the tree.

First, setup the ROOT system.

In [1]:
import ROOT as R
import numpy as np
%jsroot on

Welcome to JupyROOT 6.27/01


### Load libMiniDst
We want to load the compiled library for the base class "MiniDst". This is done with the ROOT command below, which will load this into the ROOT system. The MiniDst class is then accessible to Python through the ROOT binding.

We also want to point to the directory where the "hps-analysis" source code is installed in a variable "hps_analysis_src", so we can load the helper Python code from there. Finally, we set "data_dir" to the location of the data files. 

Each of these lines below must be adjusted for your own installation of this code.

In [2]:
hps_analysis_src="/data/HPS/hps-analysis"
data_dir="/data/HPS/Analysis/Photons"
R.gSystem.Load("/data/HPS/lib/libMiniDst")

0

### Add some data

Start a TChain, and add some data files to it. In this case we would want a MC file, so that the mc_particle information is included.

In [3]:
dch = R.TChain("MiniDST")                 # Define a TChain for MiniDst data.
dch.Add(data_dir+"/pi0_455GeV_00*.root")    # Add a file. (you probably need to change the path to it.)
print(f"Loaded {dch.GetEntries():,} events.") # Print the number of events in the files loaded.

Loaded 384,250 events.


### Setup the MiniDst class

To make the data easy to look at, we need to create an instance of the MiniDst class and then link it to the data file. Before we do the linking, we want to tell the MiniDst class that we also want to read the mc_particle information in the file.

Note: If you look in the MiniDst class, there are a number of switches that turn on or off sets of leaves on the tree. Turning off parts you don't need allow for faster reading of the data.


In [4]:
mdst = R.MiniDst() 
mdst.use_mc_particles=True
mdst.DefineBranchMap()
mdst.SetBranchAddressesOnTree(dch)

### Inspecting a single event

Now we can look at what is on the tree. To inspect a single event, you can load it from the tree (any arbitrary event number) and look at the data that is inside.

Here I grab event 100, and print a few of the leaves.

Note that the total momentum of the particle is not available, only the (x,y,z) components, so we compute the total momentum.

In [5]:
dch.GetEntry(99)
print("part type:" + str(list(mdst.part.type)))
print("part pdg :" + str(list(mdst.part.pdg)))
print("part E   :" + str(list(mdst.part.energy)))
print("part mom :[",end="")
for i in range(len(mdst.part.px)):
    print(np.sqrt(mdst.part.px[i]**2+mdst.part.py[i]**2+mdst.part.pz[i]**2),end=", ")
print("]")
print("clus E   :" + str(list(mdst.ecal_cluster_energy)))

part type:[0, 9]
part pdg :[11, 11]
part E   :[4.3694586753845215, 4.3694586753845215]
part mom :[4.557879567741962, 4.600983465185292, ]
clus E   :[4.3694586753845215]


### Show the MC particles.

The initial MC particles in the simulation are likely to interact with material in the target or the detector and produce other particles. To print the complete list of this shower, there are some helper function in the Python file "root_helpers.py" in the "hps-analysis/Python" directory. 

Here we load the helper functions and use them to print the particle shower.

In [6]:
import sys
sys.path.append(hps_analysis_src+"/Python")  # Tell Python where the root_helpers.py file is.
from root_helpers import print_mc_particle_tree   # Load the functions we want.
from root_helpers import print_daughters

In [7]:
# Print the MC tree for the current event:
# Information for each particle is: item # in mc_particle list, PDG particle number,  Energy, (px, py, pz)
# Daughter particles are shown by a | line and then the information about the daughters indented.
print_mc_particle_tree(mdst)

   8  pdg: 2212  E:  0.945721 p = (-0.078711, 0.039382, 0.080139)
  12  pdg:   11  E:  4.228935 p = ( 0.240077,-0.171737, 4.218621)
              | 
             34  pdg:   22  E:  0.000000 p = (-0.000000,-0.000000, 0.000000)
                         | 
                        18  pdg:   11  E:  0.000511 p = (-0.000000, 0.000000,-0.000000)
                                    | 
                                   31  pdg:   22  E:  0.000000 p = (-0.000000,-0.000000, 0.000000)
                                               | 
                                              15  pdg:   11  E:  0.000511 p = ( 0.000000, 0.000000,-0.000000)
                                                          | 
                                                          9  pdg:   22  E:  0.001835 p = ( 0.001071, 0.000964,-0.001136)
                                                                     | 
                                                                    33  pdg:   11  E:  0.002070 p = ( 0.00

## Simple Loops through events.

Here we loop through 1000 events and if the event has a vertex, print the vertex type. 
The vertex type numbers are defined in MiniDst.h 

In [8]:
from root_helpers import get_vertex_dictionary
vdict = get_vertex_dictionary()
for i in range(1000):
    dch.GetEntry(i)   # Get the event
    if len(mdst.v0.type) > 0:
        print(f"{i:5d} - [",end="")
        for v in mdst.v0.type:
            print(f"{v:2d} = {vdict[v]}", end=", ") # Print the vertex types for all vertexes in the event.
        print("]")
#
# The nomenclature here: 
# UC = unconstrained, TC = target constrained, BSC = beam spot constrained
# V0 = is a vertex, VC is a vertex candidate,  Moller is an e- e- vertex.
# KF = vertex from Kalman Fitter tracks,  GBL = vertex from GBL tracks.

   73 - [ 8 = UC_VC_VERTICES_KF, ]
   82 - [ 3 = TC_V0_VERTICES_KF, 12 = TC_V0_VERTICES_GBL, ]
  213 - [ 3 = TC_V0_VERTICES_KF, 12 = TC_V0_VERTICES_GBL, ]
  256 - [ 3 = TC_V0_VERTICES_KF, 12 = TC_V0_VERTICES_GBL, ]
  322 - [ 3 = TC_V0_VERTICES_KF, 12 = TC_V0_VERTICES_GBL, ]
  413 - [ 8 = UC_VC_VERTICES_KF, 17 = UC_VC_VERTICES_GBL, ]
  483 - [ 8 = UC_VC_VERTICES_KF, 17 = UC_VC_VERTICES_GBL, ]
  540 - [ 3 = TC_V0_VERTICES_KF, 12 = TC_V0_VERTICES_GBL, ]
  550 - [ 3 = TC_V0_VERTICES_KF, 12 = TC_V0_VERTICES_GBL, ]
  898 - [12 = TC_V0_VERTICES_GBL, ]
  948 - [ 8 = UC_VC_VERTICES_KF, ]


### Book a histogram and fill with a loop.

We can also create a histogram and fill it with the events from the file. This is not as efficient as a RDataFrame, but it
can be conceptually and programaticall easier.

Here we fill a histogram with the ECal energy for particles that were identified as photons. We also check that for all particles
with a valid ECal cluster associated to it, the energy for that particle is the same as the energy of the Ecal cluster that the
particle points to. These should be the same if all the code is working properly.


In [9]:
hh = R.TH1F("hh","Ecal energy for photons",1000, 0., 5.);  # Create a histogram
num_clus_checked=0
for ievt in range(dch.GetEntries()):  #  For all the events in the chain.
    dch.GetEntry(ievt)                # Get the event.
    for ipart in range(len(mdst.part.type)):              # Go through all the particles.
        if mdst.part.pdg[ipart] == 22:              # If it is a photon.
            hh.Fill( mdst.part.energy[ipart])       # Add the energy to the histogram
            
        # For any tpy particle check if the part.energy is the same as the energy of the ecal cluster.
        i_clus = mdst.part.ecal_cluster[ipart]
        if i_clus>0:
            num_clus_checked += 1
            if np.abs(mdst.ecal_cluster_energy[i_clus] - mdst.part.energy[ipart]) > 1E-6:
                print(f"Not the same: [{i_clus}] {mdst.ecal_cluster_energy[i_clus]}  [{ipart}]{mdst.part.energy[ipart]} ")

print(f"Checked {num_clus_checked} clusters.")

# Draw the histogram we filled on a canvas.
canv = R.TCanvas("canv","Demo Canvas", 800, 600)
hh.Draw()
canv.Draw()

Checked 68370 clusters.
